In [1]:
from pathlib import Path # reads paths in the current OS
import unicodedata
import pandas as pd
import numpy as np
import json
import re
import utils as ut
from stop_words import get_stop_words
from unidecode import unidecode

## Reading

In [2]:
config       = json.load(open('paths.cfg'))

data=pd.read_json(Path(config['input_path'])/'db_mesinfo.json', orient='index').fillna('')
locations_df = pd.read_csv(Path(config['input_path']) /  'municipis_merge.csv').fillna('')

payment_cat  = json.load(open('payment_list.cfg'))
product_cat  = json.load(open('product_list.cfg'))


print(data.shape)

data.rename(columns={'url':'URL', 
                     'Nom de la persona productora:':'PRODUCTOR', 
                     'Marca:':'MARCA', 
                     'Municipi:':'MUNICIPIO',  
                     'On serveix:':'DONDE', 
                     'Productes disponibles:':'PRODUCTOS', 
                     'Altres productes alimentaris:':'OTROS',
                     'Possibilitats pagament:': 'PAGAMENTO', 
                     'Fruita*:':'FRUTA', 
                     '*':'NOTAS', 
                     'Més informació:':'INFO', 
                     'Carn:':'CARNE', 
                     'Verdura*:': 'VERDURA', 
                     'Flor i planta ornamental:':'FLORES'}, inplace=True)

data.head()


(374, 14)


,URL,PRODUCTOR,MARCA,MUNICIPIO,DONDE,PRODUCTOS,OTROS,PAGAMENTO,INFO,VERDURA,NOTAS,FRUTA,CARNE,FLORES
piteus-vins-vermuts-i-cava,https://pagesiaacasa.cat/productor/piteus-vins...,"PITEUS vins, vermuts i cava",PITEUS,Corbera del Llobregat (Baix Llobregat),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Vi,"Efectiu, Transferència prèvia, Targeta",,,,,,
jordi-riera-flores,https://pagesiaacasa.cat/productor/jordi-riera...,Jordi Riera Flores,Molí de Rué,Vinebre (Ribera d’Ebre),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,"Oli, Vi i caves","Efectiu, Bizum, Transferència prèvia, Targeta",<p>Vins DOP Tarragona</p>,,,,,
agusti-segranyes-i-carulla,https://pagesiaacasa.cat/productor/agusti-segr...,Agustí Segranyes i Carulla (Producció ecològica),La Cabana d'Alcover SL,Ponts (Noguera),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Oli,"Efectiu, Transferència prèvia",,,,,,
pilar-comes,https://pagesiaacasa.cat/productor/pilar-comes,Pilar Comes (Producció ecològica),PARC DE LES OLORS DEL SERRAT,SANTA EULÀLIA DE RONÇANA (Vallès Oriental),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Herbes Aromàtiques,Transferència prèvia,,"Bledes, Calçots, Carxofes, Enciams, Escaroles,...",*Disponibilitat segons temporada,,,
sergi-martin-garcia,https://pagesiaacasa.cat/productor/sergi-marti...,Sergi Martín García (Producció integrada),COLLITA DE L'AVI PIÀ,Valls (Alt Camp),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Oli,"Efectiu, Bizum, Transferència prèvia, Targeta",,"Cebes, Tomàquets",*Disponibilitat segons temporada,,,


In [3]:
#load a set of stop words
stopwords = get_stop_words('catalan')
#add new stopwords
newStopWords = ['que','des', 'al', 'del', 'ho', 'd', 'l','per','tambe', 'fins',
               'a', 'cap', 'hi', 'ni', 'no']
stopwords.extend(newStopWords)

for col in ['OTROS', 'PAGAMENTO']:
    data[col] = data[col].apply(lambda x: ut.pre_process(x, stopwords,sw=True))


In [5]:
#data.head(10)

## Binary variables

In [6]:
# create binary variables representing whether they have a payment method or not
for key, val in payment_cat.items():
    data[key]=0
    data.loc[data['PAGAMENTO'].str.contains(r'\b'+'|'.join(val)+r'\b'),key] = 1
    
# create binary variables representing whether they have a type of product or not
for key, val in product_cat.items():
    data[key]=0
    data.loc[data['OTROS'].str.contains(r'\b'+'|'.join(val)+r'\b'),key] = 1

        
        

## Updating the columns vegetables, fruit, meat and flowers, 
## since in this dataset are informed in different fields (and not in the main one OTROS)
data.loc[data.VERDURA != '', 'vegetables']=1
data.loc[data.FRUTA != '', 'fruit']=1
data.loc[data.CARNE != '', 'meat']=1
data.loc[data.FLORES != '', 'flowers']=1



In [ ]:
#data.columns
#data[['OTROS','VERDURA', 'CARNE', 'FRUTA','vegetables','meat','fruit','charcuterie','fruit_veggies_products', 'bread_pastries', 'alcohol']].head(10)

## Integer variables

In [7]:
# Creating variables about number of products sold:
data['n_prod_princ'] = data['meat'] + data['fruit'] + data['vegetables']
data['n_prod_others'] = data['flowers'] + data['legumes'] + data['charcuterie']+ data['mushrooms'] + data['rice'] +\
    data['flour_cereals'] + data['oil_olives_vinager'] + data['eggs'] + data['dairies'] +\
    data['herbs_spices'] + data['hygiene_medicines'] + data['alcohol'] +\
    data['fruit_veggies_products'] + data['drinks'] + data['bread_pastries'] +\
    data['pasta'] + data['others']

data['n_prod_tot'] = data['n_prod_princ'] + data['n_prod_others']


# Creating variable about number of payment methods:
data['N_FORMAS_PAG']=data.PAG_BIZUM+data.PAG_EFECTIU+data.PAG_TARGETA+data.PAG_TRANSF

# Creating variable about number of comarcas where they deliver:
data['N_COMARCAS_ENTREGA']=data.DONDE.str.count(',')+1 

## 42 comarcas en total:   
    
#Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell, Alta Ribagorça, 
#Anoia, Bages,Baix Camp, Baix Ebre, Baix Empordà,
#Baix Llobregat, Baix Penedès, Barcelonès, Berguedà, Cerdanya, 
#Conca de Barberà, Garraf, Garrigues, Garrotxa, Gironès,
#Maresme, Moianès, Montsià, Noguera, Osona / Lluçanès, 
#Pallars Jussà, Pallars Sobirà, Pla d'Urgell, Pla de l'Estany, Priorat, 
#Ribera d'Ebre, Ripollès, Segarra, Segrià, Selva,
#Solsonès, Tarragonès, Terra Alta, Urgell, Vall d'Aran, 
#Vallès Occidental,  Vallès Oriental

In [ ]:
#data.loc[data.CARNE.isna(), ['PRODUCTOR', 'MUNICIPIO','CARNE', 'FRUTA', 'VERDURA', 'OTROS']]
#data.loc[data.CARNE.isna() & data.FRUTA.isna() & data.VERDURA.isna() & data.OTROS.isna(), ['PRODUCTOR', 'MUNICIPIO','INFO','CARNE', 'FRUTA', 'VERDURA', 'OTROS']]
##7 prodctores no tienen ni carne, ni fruta, ni verdura ni otros productos

#data.loc[data.PRODUCTOR.str.contains('ANA GAYOSO')] #venden shampoo etc. (lo dice en INFO_EXTRA)
#data[data.INFO.notna()].shape ## 177 (197 no tienen info)
##data.loc[data.INFO.notna(), ['PRODUCTOR', 'MUNICIPIO','INFO','CARNE', 'FRUTA', 'VERDURA', 'OTROS']]

#data.INFO.unique()
#data.groupby(['INFO']).agg({'PRODUCTOR': ['count']})
#data.INFO.head(10)

In [8]:
data.to_csv(Path(config['input_path']) / 'pagesos_clean.csv', index=False)